# Plant Disease Monitoring System – Premium Edition 🌱
## Cloud Computing Assignment - HW2
### Braude Academic College of Engineering

This system implements a comprehensive plant disease monitoring solution with RAG-powered search engine, IoT sensors, and visual dashboards.

**Adafruit IO Credentials:**
- Username: braude1
- AIO Key: aio_mUTZ65zfctMaN1DD7CdPWpB4PRJM
- Feeds: temperature, humidity, soil, json

## 📦 Installation & Setup

In [ ]:
# Install required packages
!pip install -q gradio==4.* sentence-transformers chromadb scikit-learn
!pip install -q adafruit-io paho-mqtt requests plotly pandas numpy pillow
!pip install -q nltk gdown

print("✓ All packages installed successfully")

In [ ]:
# -*- coding: utf-8 -*-
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
import re
import time
import warnings
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import io

warnings.filterwarnings('ignore')

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("✓ Core libraries imported")

In [ ]:
# Package detection
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    CHROMADB_AVAILABLE = False

try:
    from sentence_transformers import SentenceTransformer
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False

try:
    import gradio as gr
    GRADIO_AVAILABLE = True
except ImportError:
    GRADIO_AVAILABLE = False

try:
    from Adafruit_IO import Client, RequestError, Feed
    ADAFRUIT_AVAILABLE = True
except ImportError:
    ADAFRUIT_AVAILABLE = False

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print(f"ChromaDB: {'✓' if CHROMADB_AVAILABLE else '✗'}")
print(f"Transformers: {'✓' if TRANSFORMERS_AVAILABLE else '✗'}")
print(f"Gradio: {'✓' if GRADIO_AVAILABLE else '✗'}")
print(f"Adafruit IO: {'✓' if ADAFRUIT_AVAILABLE else '✗'}")

## 🔧 Vector Store & RAG System

In [ ]:
class SimpleVectorStore:
    """Lightweight vector store for when ChromaDB is unavailable"""

    def __init__(self):
        self.documents = []
        self.embeddings = []
        self.metadatas = []
        self.ids = []

    def add(self, embeddings, documents, metadatas, ids):
        self.embeddings.extend(embeddings)
        self.documents.extend(documents)
        self.metadatas.extend(metadatas)
        self.ids.extend(ids)

    def query(self, query_embeddings, n_results=5):
        if not self.embeddings:
            return {'ids': [[]], 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}

        similarities = cosine_similarity(query_embeddings, self.embeddings)[0]
        top_indices = np.argsort(similarities)[::-1][:n_results]

        return {
            'ids': [[self.ids[i] for i in top_indices]],
            'documents': [[self.documents[i] for i in top_indices]],
            'metadatas': [[self.metadatas[i] for i in top_indices]],
            'distances': [[1 - similarities[i] for i in top_indices]]
        }

    def count(self):
        return len(self.documents)

print("✓ SimpleVectorStore ready")

In [ ]:
class PlantDiseaseRAG:
    """Main RAG system for plant disease research papers"""

    def __init__(self):
        self._initialize_components()
        self.papers = []
        self.fitted = False

    def _initialize_components(self):
        """Initialize all system components"""
        # Setup embedding model
        if TRANSFORMERS_AVAILABLE:
            try:
                self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
                self.use_transformers = True
                print("✓ Using Sentence Transformers for embeddings")
            except Exception:
                self._setup_tfidf()
        else:
            self._setup_tfidf()

        # Setup vector store
        if CHROMADB_AVAILABLE:
            try:
                client = chromadb.Client()
                try:
                    self.collection = client.get_collection("plant_disease_papers")
                except Exception:
                    self.collection = client.create_collection("plant_disease_papers")
                self.use_chromadb = True
                print("✓ Using ChromaDB for vector storage")
            except Exception:
                self.collection = SimpleVectorStore()
                self.use_chromadb = False
        else:
            self.collection = SimpleVectorStore()
            self.use_chromadb = False
            print("✓ Using SimpleVectorStore for vector storage")

    def _setup_tfidf(self):
        """Setup TF-IDF as fallback"""
        self.use_transformers = False
        self.tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
        print("✓ Using TF-IDF for embeddings")

    def preprocess_text(self, text):
        """Clean and prepare text for processing"""
        if not text:
            return ""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s\-\.\(\)]', ' ', text)
        return text.strip()

    def extract_entities(self, text):
        """Extract plant disease related entities from text"""
        entities = {'diseases': [], 'plants': [], 'methods': []}

        # Diseases
        diseases = re.findall(
            r'\b(blight|rust|mildew|rot|wilt|spot|canker|mosaic|virus|fungal|bacterial)\b',
            text, re.IGNORECASE
        )
        entities['diseases'] = list(set(diseases))[:5]

        # Plants
        plants = re.findall(
            r'\b(tomato|potato|wheat|corn|rice|apple|grape|citrus|plant|crop|leaf)\b',
            text, re.IGNORECASE
        )
        entities['plants'] = list(set(plants))[:5]

        # Methods
        methods = re.findall(
            r'\b(CNN|deep learning|machine learning|image processing|detection|classification|neural network|IoT|sensor)\b',
            text, re.IGNORECASE
        )
        entities['methods'] = list(set(methods))[:5]

        return entities

    def generate_embeddings(self, texts):
        """Generate embeddings using available method"""
        if self.use_transformers:
            return self.embedding_model.encode(texts, show_progress_bar=False)
        else:
            if not self.fitted:
                self.tfidf.fit(texts)
                self.fitted = True
            return self.tfidf.transform(texts).toarray()

    def load_papers(self, papers_data):
        """Load papers into the RAG system"""
        valid_papers = [p for p in papers_data if p.get('content', '').strip()]

        if not valid_papers:
            return False

        documents, metadatas, ids = [], [], []

        for i, paper in enumerate(valid_papers):
            text = f"{paper.get('title', '')} {paper.get('content', '')}"
            text = self.preprocess_text(text)

            if len(text) < 50:
                continue

            entities = self.extract_entities(text)

            metadata = {
                'title': paper.get('title', 'Unknown'),
                'doc_id': paper.get('doc_id', f'doc{i+1}'),
                'diseases': ', '.join(entities['diseases']),
                'plants': ', '.join(entities['plants']),
                'methods': ', '.join(entities['methods'])
            }

            documents.append(text)
            metadatas.append(metadata)
            ids.append(f"paper_{i}")

        if not documents:
            return False

        # Generate embeddings
        embeddings = self.generate_embeddings(documents)

        # Add to vector store
        if getattr(self, 'use_chromadb', False):
            try:
                self.collection.add(
                    embeddings=embeddings.tolist(),
                    documents=documents,
                    metadatas=metadatas,
                    ids=ids
                )
            except Exception:
                self.collection = SimpleVectorStore()
                self.collection.add(
                    embeddings=embeddings,
                    documents=documents,
                    metadatas=metadatas,
                    ids=ids
                )
        else:
            self.collection.add(
                embeddings=embeddings,
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )

        self.papers = valid_papers
        return True

    def search(self, query, n_results=3):
        """Search for relevant papers"""
        query_processed = self.preprocess_text(query)
        query_embedding = self.generate_embeddings([query_processed])

        if getattr(self, 'use_chromadb', False):
            results = self.collection.query(
                query_embeddings=query_embedding.tolist(),
                n_results=n_results
            )
        else:
            results = self.collection.query(
                query_embeddings=query_embedding,
                n_results=n_results
            )

        return results

    def generate_response(self, query, search_results):
        """Generate response based on search results"""
        if not search_results['documents'][0]:
            return "No relevant papers found for your query."

        papers = search_results['metadatas'][0]
        documents = search_results['documents'][0]
        distances = search_results['distances'][0]

        response = f"## 🔍 Research Findings for: {query}\n\n"

        for i, (paper, doc, dist) in enumerate(zip(papers, documents, distances)):
            score = (1 - dist) * 100
            response += f"### {i+1}. {paper['title']}\n"
            response += f"**Relevance Score:** {score:.1f}%\n"
            response += f"**Document ID:** {paper['doc_id']}\n"

            if paper.get('diseases'):
                response += f"**Diseases:** {paper['diseases']}\n"
            if paper.get('plants'):
                response += f"**Plants:** {paper['plants']}\n"
            if paper.get('methods'):
                response += f"**Methods:** {paper['methods']}\n"

            # Preview
            preview = doc[:300] + "..." if len(doc) > 300 else doc
            response += f"\n**Preview:** {preview}\n\n"
            response += "---\n\n"

        # Add summary
        all_diseases = set()
        all_plants = set()
        all_methods = set()
        for paper in papers:
            if paper.get('diseases'):
                all_diseases.update([d.strip() for d in paper['diseases'].split(',') if d.strip()])
            if paper.get('plants'):
                all_plants.update([p.strip() for p in paper['plants'].split(',') if p.strip()])
            if paper.get('methods'):
                all_methods.update([m.strip() for m in paper['methods'].split(',') if m.strip()])

        response += "### 📊 Summary\n"
        if all_diseases:
            response += f"**Key Diseases:** {', '.join(list(all_diseases)[:5])}\n"
        if all_plants:
            response += f"**Key Plants:** {', '.join(list(all_plants)[:5])}\n"
        if all_methods:
            response += f"**Methods Used:** {', '.join(list(all_methods)[:5])}\n"

        return response

    def query(self, question, n_results=5):
        """Main query function"""
        search_results = self.search(question, n_results)
        response = self.generate_response(question, search_results)

        return {
            'question': question,
            'response': response,
            'papers_found': len(search_results['documents'][0]),
            'search_results': search_results
        }

    def get_status(self):
        """Get system status"""
        return {
            'vector_db': 'ChromaDB' if getattr(self, 'use_chromadb', False) else 'Simple Store',
            'embeddings': 'Transformer' if getattr(self, 'use_transformers', False) else 'TF-IDF',
            'papers_loaded': len(self.papers) if self.papers else 0
        }

print("✓ PlantDiseaseRAG class ready")

## 📚 Plant Disease Documents

In [ ]:
def get_plant_disease_papers():
    """Get plant disease research papers for RAG system"""
    return [
        {
            'doc_id': 'doc1',
            'title': 'Deep Learning for Plant Disease Detection',
            'content': '''Plant disease detection using deep learning has shown remarkable success in recent years.
            Convolutional neural networks can identify various plant diseases with high accuracy.
            Image classification techniques enable early disease detection in crops like tomatoes, potatoes, and wheat.
            The integration of computer vision and machine learning provides automated disease diagnosis.
            Farmers can use mobile applications to detect diseases in real-time and prevent crop loss.
            Deep learning models trained on large datasets of diseased plant images achieve over 95% accuracy.
            Transfer learning from pre-trained models like ResNet and VGG accelerates model development.
            Real-time inference on edge devices enables field deployment of disease detection systems.'''
        },
        {
            'doc_id': 'doc2',
            'title': 'Fungal Infections in Agricultural Plants',
            'content': '''Fungal infections are major threats to agricultural productivity worldwide.
            Common fungal diseases include powdery mildew, rust, and blight affecting various crops.
            Early detection and treatment are crucial for preventing widespread infection.
            Symptoms include discoloration, spots, and wilting of plant tissues.
            Integrated pest management combines cultural, biological, and chemical control methods.
            Fungicides should be applied preventively during high-risk periods.
            Crop rotation and resistant varieties help reduce fungal disease pressure.
            Environmental conditions like humidity and temperature significantly influence fungal spread.'''
        },
        {
            'doc_id': 'doc3',
            'title': 'IoT-Based Plant Health Monitoring Systems',
            'content': '''Internet of Things sensors enable continuous monitoring of plant health parameters.
            Temperature, humidity, and soil moisture sensors provide real-time data for disease prediction.
            MQTT protocol facilitates efficient data transmission from sensors to cloud platforms.
            Machine learning algorithms analyze sensor data to detect disease patterns.
            Automated irrigation and fertilization systems respond to sensor readings.
            Edge computing reduces latency for time-critical plant health decisions.
            Wireless sensor networks cover large agricultural areas efficiently.
            Integration with weather data improves disease outbreak prediction accuracy.'''
        },
        {
            'doc_id': 'doc4',
            'title': 'Image Processing Techniques for Leaf Disease Classification',
            'content': '''Image processing plays a vital role in automated plant disease diagnosis.
            Segmentation algorithms isolate diseased regions from healthy plant tissue.
            Feature extraction methods capture texture, color, and shape characteristics.
            Support vector machines and random forests classify disease types accurately.
            Mobile imaging applications make disease detection accessible to farmers worldwide.
            Hyperspectral imaging reveals disease symptoms before visible signs appear.
            Data augmentation techniques improve model generalization on limited datasets.
            Multi-scale analysis captures both local lesions and global disease patterns.'''
        },
        {
            'doc_id': 'doc5',
            'title': 'Climate Change Impact on Plant Disease Patterns',
            'content': '''Climate change significantly affects plant disease distribution and severity.
            Warmer temperatures and altered precipitation patterns favor certain pathogens.
            Bacterial and viral infections increase under stress conditions.
            Monitoring environmental factors helps predict disease outbreaks.
            Adaptive agricultural practices are necessary to mitigate climate-related disease risks.
            New pest and disease threats emerge as geographic ranges shift.
            Drought stress weakens plant immune responses to pathogen attacks.
            Integrated climate-disease models guide long-term agricultural planning.'''
        }
    ]

print(f"✓ {len(get_plant_disease_papers())} plant disease papers available")

## 🌡️ IoT Sensor System

In [ ]:
# Adafruit IO Configuration
ADAFRUIT_IO_USERNAME = "braude1"
ADAFRUIT_IO_KEY = "aio_mUTZ65zfctMaN1DD7CdPWpB4PRJM"

aio = None
if ADAFRUIT_AVAILABLE:
    try:
        aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)
        print("✓ Connected to Adafruit IO")
        feeds = aio.feeds()
        print(f"📡 Available feeds: {[f.key for f in feeds]}")
    except Exception as e:
        print(f"⚠️ Adafruit IO connection error: {e}")
        aio = None
else:
    print("⚠️ Adafruit IO not available")

In [ ]:
def get_latest_sensor_data():
    """Get the latest sensor readings"""
    if not aio:
        # Return simulated data if Adafruit IO not available
        return {
            'temperature': 24.5 + np.random.randn() * 2,
            'humidity': 65.0 + np.random.randn() * 5,
            'soil': 45.0 + np.random.randn() * 5,
            'timestamp': datetime.now()
        }
    try:
        temp_data = aio.receive('temperature')
        humidity_data = aio.receive('humidity')
        soil_data = aio.receive('soil')

        return {
            'temperature': float(temp_data.value),
            'humidity': float(humidity_data.value),
            'soil': float(soil_data.value),
            'timestamp': datetime.now()
        }
    except Exception as e:
        print(f"Error reading sensors: {e}")
        return None

def get_historical_data(feed_key, hours=24):
    """Get historical sensor data"""
    if not aio:
        # Generate simulated historical data
        timestamps = pd.date_range(end=datetime.now(), periods=hours*2, freq='30min')
        base_values = {'temperature': 24, 'humidity': 65, 'soil': 45}
        values = [base_values.get(feed_key, 50) + np.random.randn() * 3 for _ in timestamps]
        return pd.DataFrame({'timestamp': timestamps, 'value': values})
    
    try:
        data = aio.data(feed_key)
        df = pd.DataFrame([
            {
                'value': float(d.value),
                'timestamp': pd.to_datetime(d.created_at)
            }
            for d in data
        ])
        cutoff = pd.Timestamp.now(tz='UTC') - timedelta(hours=hours)
        df = df[df['timestamp'] > cutoff]
        return df.sort_values('timestamp')
    except Exception as e:
        print(f"Error getting historical data: {e}")
        return pd.DataFrame()

print("✓ Sensor functions ready")

## ⚡ Anomaly Detection System

In [ ]:
class PlantHealthMonitor:
    """Advanced anomaly detection for plant health monitoring"""

    def __init__(self):
        self.alert_history = []
        self.thresholds = {
            'temperature': {'min': 15, 'max': 32, 'optimal_min': 18, 'optimal_max': 28},
            'humidity': {'min': 40, 'max': 95, 'optimal_min': 60, 'optimal_max': 80},
            'soil': {'min': 20, 'max': 80, 'optimal_min': 40, 'optimal_max': 60}
        }

    def check_anomalies(self, sensor_data):
        """Check for anomalies in sensor readings"""
        alerts = []

        for sensor, value in sensor_data.items():
            if sensor == 'timestamp':
                continue

            if sensor in self.thresholds:
                threshold = self.thresholds[sensor]

                if value < threshold['min']:
                    alerts.append({
                        'severity': 'CRITICAL',
                        'sensor': sensor,
                        'value': value,
                        'message': f'{sensor.capitalize()} critically low: {value:.1f}',
                        'recommendation': f'Immediate action required! Below safe minimum ({threshold["min"]})'
                    })
                elif value > threshold['max']:
                    alerts.append({
                        'severity': 'CRITICAL',
                        'sensor': sensor,
                        'value': value,
                        'message': f'{sensor.capitalize()} critically high: {value:.1f}',
                        'recommendation': f'Immediate action required! Exceeds safe maximum ({threshold["max"]})'
                    })
                elif value < threshold['optimal_min']:
                    alerts.append({
                        'severity': 'WARNING',
                        'sensor': sensor,
                        'value': value,
                        'message': f'{sensor.capitalize()} below optimal: {value:.1f}',
                        'recommendation': f'Consider adjusting. Optimal range: {threshold["optimal_min"]}-{threshold["optimal_max"]}'
                    })
                elif value > threshold['optimal_max']:
                    alerts.append({
                        'severity': 'WARNING',
                        'sensor': sensor,
                        'value': value,
                        'message': f'{sensor.capitalize()} above optimal: {value:.1f}',
                        'recommendation': f'Consider adjusting. Optimal range: {threshold["optimal_min"]}-{threshold["optimal_max"]}'
                    })

        if alerts:
            for alert in alerts:
                alert['timestamp'] = datetime.now()
                self.alert_history.append(alert)

        return alerts

    def get_health_score(self, sensor_data):
        """Calculate overall plant health score"""
        scores = []
        for sensor in ['temperature', 'humidity', 'soil']:
            if sensor in sensor_data:
                value = sensor_data[sensor]
                threshold = self.thresholds[sensor]
                
                if threshold['optimal_min'] <= value <= threshold['optimal_max']:
                    scores.append(100)
                elif threshold['min'] <= value < threshold['optimal_min']:
                    scores.append(70)
                elif threshold['optimal_max'] < value <= threshold['max']:
                    scores.append(70)
                else:
                    scores.append(30)
        
        return sum(scores) / len(scores) if scores else 0

# Initialize monitor
health_monitor = PlantHealthMonitor()
print("✓ Plant Health Monitor initialized")

## 📊 Dashboard Visualization

In [ ]:
def create_sensor_dashboard(hours=24):
    """Create comprehensive dashboard with sensor data visualizations"""
    # Get historical data for all sensors
    temp_df = get_historical_data('temperature', hours)
    humidity_df = get_historical_data('humidity', hours)
    soil_df = get_historical_data('soil', hours)

    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('🌡️ Temperature Trend', '💧 Humidity Trend',
                       '🌱 Soil Moisture Trend', '❤️ Health Score'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
               [{'type': 'scatter'}, {'type': 'indicator'}]]
    )

    # Temperature plot
    if not temp_df.empty:
        fig.add_trace(
            go.Scatter(x=temp_df['timestamp'], y=temp_df['value'],
                      mode='lines+markers', name='Temperature',
                      line=dict(color='#ef4444', width=2),
                      marker=dict(size=4)),
            row=1, col=1
        )

    # Humidity plot
    if not humidity_df.empty:
        fig.add_trace(
            go.Scatter(x=humidity_df['timestamp'], y=humidity_df['value'],
                      mode='lines+markers', name='Humidity',
                      line=dict(color='#3b82f6', width=2),
                      marker=dict(size=4)),
            row=1, col=2
        )

    # Soil moisture plot
    if not soil_df.empty:
        fig.add_trace(
            go.Scatter(x=soil_df['timestamp'], y=soil_df['value'],
                      mode='lines+markers', name='Soil Moisture',
                      line=dict(color='#22c55e', width=2),
                      marker=dict(size=4)),
            row=2, col=1
        )

    # Current status gauge
    current = get_latest_sensor_data()
    if current:
        health_score = health_monitor.get_health_score(current)

        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=health_score,
                title={'text': "Plant Health", 'font': {'color': '#f8fafc'}},
                delta={'reference': 80},
                gauge={
                    'axis': {'range': [None, 100], 'tickcolor': '#475569'},
                    'bar': {'color': "#22c55e"},
                    'bgcolor': '#1e293b',
                    'steps': [
                        {'range': [0, 50], 'color': "#ef4444"},
                        {'range': [50, 75], 'color': "#eab308"},
                        {'range': [75, 100], 'color': "#22c55e"}
                    ],
                    'threshold': {
                        'line': {'color': "white", 'width': 4},
                        'thickness': 0.75,
                        'value': 90
                    }
                }
            ),
            row=2, col=2
        )

    # Update layout with dark theme
    fig.update_layout(
        title_text="🌱 Plant Health Monitoring Dashboard",
        title_font_color='#f8fafc',
        showlegend=False,
        height=700,
        font=dict(size=11, color='#f8fafc'),
        paper_bgcolor='#0f172a',
        plot_bgcolor='#1e293b'
    )

    # Update axes
    for i in range(1, 3):
        for j in range(1, 3):
            if not (i == 2 and j == 2):
                fig.update_xaxes(gridcolor='#334155', row=i, col=j)
                fig.update_yaxes(gridcolor='#334155', row=i, col=j)

    fig.update_xaxes(title_text="Time", title_font_color='#94a3b8', row=1, col=1)
    fig.update_xaxes(title_text="Time", title_font_color='#94a3b8', row=1, col=2)
    fig.update_xaxes(title_text="Time", title_font_color='#94a3b8', row=2, col=1)
    fig.update_yaxes(title_text="°C", title_font_color='#94a3b8', row=1, col=1)
    fig.update_yaxes(title_text="%", title_font_color='#94a3b8', row=1, col=2)
    fig.update_yaxes(title_text="%", title_font_color='#94a3b8', row=2, col=1)

    return fig

print("✓ Dashboard function ready")

## 🎨 Premium Gradio Interface

In [ ]:
def create_premium_interface(rag_system):
    """Create elegant Gradio web interface with dark theme"""

    import gradio as gr

    # --- RAG Search Functions ---
    def query_papers(question, n_results):
        if not question.strip():
            return "Please enter a research question to get started."
        result = rag_system.query(question, n_results=int(n_results))
        return result['response']

    def get_system_metrics():
        status = rag_system.get_status()
        return f"""
**System Configuration**

🗄️ Vector Database: {status['vector_db']}
🔮 Embeddings: {status['embeddings']}
📚 Papers Loaded: {status['papers_loaded']}
        """

    # --- Sensor Functions ---
    def get_sensor_display():
        data = get_latest_sensor_data()
        if data:
            alerts = health_monitor.check_anomalies(data)
            health_score = health_monitor.get_health_score(data)
            
            output = f"""## 🌡️ Current Sensor Readings

**Timestamp:** {data['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}

| Sensor | Value | Status |
|--------|-------|--------|
| 🌡️ Temperature | {data['temperature']:.1f}°C | {'✅ Optimal' if 18 <= data['temperature'] <= 28 else '⚠️ Check'} |
| 💧 Humidity | {data['humidity']:.1f}% | {'✅ Optimal' if 60 <= data['humidity'] <= 80 else '⚠️ Check'} |
| 🌱 Soil Moisture | {data['soil']:.1f}% | {'✅ Optimal' if 40 <= data['soil'] <= 60 else '⚠️ Check'} |

**❤️ Overall Health Score: {health_score:.0f}/100**
"""
            
            if alerts:
                output += "\n### ⚠️ Alerts\n"
                for alert in alerts:
                    icon = '🔴' if alert['severity'] == 'CRITICAL' else '🟡'
                    output += f"- {icon} **{alert['severity']}:** {alert['message']}\n"
                    output += f"  - 📋 {alert['recommendation']}\n"
            else:
                output += "\n### ✅ All readings within normal range"
            
            return output
        return "❌ Unable to fetch sensor data"

    def generate_dashboard(hours):
        fig = create_sensor_dashboard(int(hours))
        return fig

    def run_anomaly_detection():
        data = get_latest_sensor_data()
        if not data:
            return "❌ Unable to fetch sensor data for analysis"
        
        alerts = health_monitor.check_anomalies(data)
        health_score = health_monitor.get_health_score(data)
        
        output = f"""## ⚡ Anomaly Detection Results

### Current Status
- 🌡️ Temperature: {data['temperature']:.1f}°C
- 💧 Humidity: {data['humidity']:.1f}%
- 🌱 Soil Moisture: {data['soil']:.1f}%

### ❤️ Health Score: {health_score:.0f}/100

"""
        
        if alerts:
            output += "### 🚨 Detected Anomalies\n\n"
            for alert in alerts:
                icon = '🔴' if alert['severity'] == 'CRITICAL' else '🟡'
                output += f"#### {icon} {alert['severity']}: {alert['sensor'].upper()}\n"
                output += f"- **Value:** {alert['value']:.1f}\n"
                output += f"- **Message:** {alert['message']}\n"
                output += f"- **Recommendation:** {alert['recommendation']}\n\n"
        else:
            output += "### ✅ No Anomalies Detected\n\nAll sensor readings are within normal parameters."
        
        return output

    def query_mqtt_data(sensor, hours):
        df = get_historical_data(sensor, int(hours))
        if df.empty:
            return "❌ No data found for the specified query"
        
        output = f"""## 📡 MQTT Query Results: {sensor.upper()}

### 📈 Statistics
- **Total Records:** {len(df)}
- **Average Value:** {df['value'].mean():.2f}
- **Min Value:** {df['value'].min():.2f}
- **Max Value:** {df['value'].max():.2f}
- **Std Deviation:** {df['value'].std():.2f}

### 📅 Time Range
- **From:** {df['timestamp'].min()}
- **To:** {df['timestamp'].max()}

### 📋 Latest 10 Readings

| Timestamp | Value |
|-----------|-------|
"""
        for _, row in df.head(10).iterrows():
            ts = row['timestamp'].strftime('%Y-%m-%d %H:%M:%S') if hasattr(row['timestamp'], 'strftime') else str(row['timestamp'])
            output += f"| {ts} | {row['value']:.2f} |\n"
        
        return output

    # Example questions
    examples = [
        ("How does deep learning detect plant diseases?", 3),
        ("What are common fungal infections in agricultural plants?", 3),
        ("How do IoT sensors monitor plant health?", 4),
        ("What image processing techniques are used for leaf disease?", 3),
        ("How does climate change affect plant diseases?", 3),
    ]

    def run_example(idx: int):
        q, n = examples[idx]
        res = rag_system.query(q, n_results=int(n))
        return q, int(n), res["response"]

    # --- Custom CSS ---
    custom_css = """
    :root {
        --primary-color: #22c55e;
        --secondary-color: #15803d;
        --background-color: #0f172a;
        --surface-color: #1e293b;
        --surface-light: #334155;
        --text-primary: #f8fafc;
        --text-secondary: #cbd5e1;
        --accent-color: #22c55e;
        --success-color: #059669;
        --border-color: #475569;
    }
    .gradio-container { background: linear-gradient(135deg, #0f172a 0%, #1e293b 100%); color: var(--text-primary); font-family: 'Inter','Segoe UI',sans-serif; min-height: 100vh; }
    .main-header { background: rgba(30,41,59,.8); backdrop-filter: blur(20px); border-radius: 16px; padding: 2rem; margin: 1rem; box-shadow: 0 8px 32px rgba(0,0,0,.3); border: 1px solid rgba(71,85,105,.3); }
    .query-section { background: rgba(30,41,59,.6); backdrop-filter: blur(15px); border-radius: 12px; padding: 1.5rem; box-shadow: 0 4px 20px rgba(0,0,0,.2); border: 1px solid rgba(71,85,105,.2); }
    .response-area { background: rgba(30,41,59,.7) !important; border-radius: 12px !important; border: 1px solid rgba(71,85,105,.3) !important; box-shadow: 0 4px 20px rgba(0,0,0,.15) !important; color: var(--text-primary) !important; }
    .status-panel { background: linear-gradient(135deg,#15803d 0%,#166534 100%); border-radius: 12px; padding: 1rem; color: white; box-shadow: 0 4px 15px rgba(34,197,94,.2); border: 1px solid rgba(134,239,172,.2); }

    .gradio-textbox textarea, .gradio-textbox input { background: rgba(51,65,85,.8) !important; border: 1px solid rgba(71,85,105,.4) !important; border-radius: 8px !important; color: var(--text-primary) !important; font-size: 14px !important; }
    .gradio-textbox textarea:focus, .gradio-textbox input:focus { border-color: var(--accent-color) !important; box-shadow: 0 0 0 2px rgba(34,197,94,.2) !important; }
    .gradio-button { background: linear-gradient(135deg,var(--primary-color) 0%,var(--secondary-color) 100%) !important; border: none !important; border-radius: 8px !important; color: white !important; font-weight: 600 !important; transition: all .3s ease !important; }
    .gradio-button:hover { transform: translateY(-1px) !important; box-shadow: 0 6px 20px rgba(34,197,94,.3) !important; }
    .gradio-slider input[type="range"] { background: rgba(51,65,85,.8) !important; }
    .gradio-accordion { background: rgba(30,41,59,.5) !important; border: 1px solid rgba(71,85,105,.3) !important; border-radius: 8px !important; }

    .examples-grid { display: grid; grid-template-columns: 1fr; gap: .5rem; }
    @media (min-width: 900px) { .examples-grid { grid-template-columns: 1fr 1fr; } }
    .example-btn { text-align: left; white-space: normal; line-height: 1.3; padding: .75rem 1rem; }
    .example-meta { opacity: .85; font-size: .9rem; }
    .markdown-content h1, .markdown-content h2, .markdown-content h3 { color: var(--text-primary) !important; }
    .markdown-content p { color: var(--text-secondary) !important; line-height: 1.6 !important; }
    .gradio-label { color: var(--text-primary) !important; font-weight: 500 !important; }

    #prompt_box textarea,
    #prompt_box input[type="text"] {
        color: black !important;
        caret-color: black !important;
    }
    #prompt_box textarea::placeholder,
    #prompt_box input[type="text"]::placeholder {
        color: #111 !important;
        opacity: .6 !important;
    }

    #docs_count input[type="number"] {
        color: #000 !important;
        -webkit-text-fill-color: #000 !important;
        caret-color: #000 !important;
        opacity: 1 !important;
    }
    
    .tab-nav button { color: var(--text-primary) !important; }
    .tab-nav button.selected { background: var(--primary-color) !important; }
    """

    with gr.Blocks(
        title="Plant Disease Monitoring System",
        theme=gr.themes.Base(
            primary_hue="green",
            secondary_hue="slate",
            neutral_hue="slate",
            font=gr.themes.GoogleFont("Inter")
        ).set(
            body_background_fill="#0f172a",
            body_text_color="#f8fafc",
            background_fill_primary="#1e293b",
            background_fill_secondary="#334155",
            border_color_primary="#475569",
            color_accent="#22c55e",
            color_accent_soft="#15803d"
        ),
        css=custom_css
    ) as interface:

        # Header
        with gr.Column(elem_classes="main-header"):
            gr.HTML("""
            <div style="text-align: center;">
                <h1 style="background: linear-gradient(45deg,#4ade80,#22c55e);
                           -webkit-background-clip: text;
                           -webkit-text-fill-color: transparent;
                           font-size: 2.5rem; margin: 0; font-weight: 700;">
                    🌱 Plant Disease Monitoring System
                </h1>
                <p style="font-size: 1.1rem; color: #94a3b8; margin-top: .5rem; font-weight: 400;">
                    AI-powered plant disease research with IoT sensor monitoring
                </p>
            </div>
            """)

        # Tabs
        with gr.Tabs():
            # Tab 1: RAG Search
            with gr.TabItem("🔍 Search Engine"):
                with gr.Row():
                    with gr.Column(scale=3, elem_classes="query-section"):
                        question_input = gr.Textbox(
                            label="🔍 Research Question",
                            placeholder="Ask about plant diseases, detection methods, or IoT monitoring...",
                            lines=3,
                            elem_id="prompt_box",
                        )
                        with gr.Row():
                            n_results_slider = gr.Slider(
                                minimum=1, maximum=5, value=3, step=1,
                                label="Papers to analyze 📄",
                                elem_id="docs_count",
                            )
                            submit_btn = gr.Button("Search Research 🚀", variant="primary")

                    with gr.Column(scale=1, elem_classes="status-panel"):
                        system_info = gr.Markdown(get_system_metrics())

                response_output = gr.Markdown(label="📊 Research Analysis", elem_classes="response-area", show_label=True)

                with gr.Accordion("💡 Example Questions", open=False):
                    gr.HTML('<div class="example-meta">Try these research questions:</div>')
                    with gr.Column(elem_classes="examples-grid"):
                        for i, (q, n) in enumerate(examples):
                            gr.Button(f"🔎 {q}", elem_classes="example-btn").click(
                                fn=lambda idx=i: run_example(idx),
                                inputs=[],
                                outputs=[question_input, n_results_slider, response_output],
                            )

                submit_btn.click(fn=query_papers, inputs=[question_input, n_results_slider], outputs=response_output)
                question_input.submit(fn=query_papers, inputs=[question_input, n_results_slider], outputs=response_output)

            # Tab 2: Sensor Data
            with gr.TabItem("🌡️ Sensor Data"):
                with gr.Row():
                    refresh_btn = gr.Button("🔄 Refresh Readings", variant="primary")
                sensor_output = gr.Markdown(elem_classes="response-area")
                refresh_btn.click(fn=get_sensor_display, inputs=[], outputs=sensor_output)
                interface.load(fn=get_sensor_display, inputs=[], outputs=sensor_output)

            # Tab 3: MQTT Query
            with gr.TabItem("📡 MQTT Query"):
                with gr.Row():
                    sensor_dropdown = gr.Dropdown(
                        choices=['temperature', 'humidity', 'soil'],
                        value='temperature',
                        label="Select Sensor"
                    )
                    hours_slider = gr.Slider(
                        minimum=1, maximum=168, value=24, step=1,
                        label="Time Range (hours)"
                    )
                    query_btn = gr.Button("🔍 Query Data", variant="primary")
                query_output = gr.Markdown(elem_classes="response-area")
                query_btn.click(fn=query_mqtt_data, inputs=[sensor_dropdown, hours_slider], outputs=query_output)

            # Tab 4: Dashboard
            with gr.TabItem("📊 Dashboard"):
                with gr.Row():
                    dashboard_hours = gr.Slider(
                        minimum=6, maximum=168, value=24, step=6,
                        label="Time Range (hours)"
                    )
                    dashboard_btn = gr.Button("📈 Generate Dashboard", variant="primary")
                dashboard_output = gr.Plot()
                dashboard_btn.click(fn=generate_dashboard, inputs=[dashboard_hours], outputs=dashboard_output)

            # Tab 5: Anomaly Detection
            with gr.TabItem("⚡ Anomaly Detection"):
                with gr.Row():
                    anomaly_btn = gr.Button("🔍 Run Anomaly Detection", variant="primary")
                anomaly_output = gr.Markdown(elem_classes="response-area")
                anomaly_btn.click(fn=run_anomaly_detection, inputs=[], outputs=anomaly_output)

            # Tab 6: Image Upload
            with gr.TabItem("🖼️ Image Upload"):
                gr.HTML("""
                <div style="text-align: center; padding: 2rem;">
                    <h3 style="color: #f8fafc;">🖼️ Plant Image Upload</h3>
                    <p style="color: #94a3b8;">Upload plant images for disease detection analysis</p>
                </div>
                """)
                image_input = gr.Image(type="pil", label="Upload Plant Image")
                image_output = gr.Markdown()
                
                def analyze_image(img):
                    if img is None:
                        return "Please upload an image to analyze."
                    return f"""## 🖼️ Image Analysis

**Image Size:** {img.size[0]} x {img.size[1]} pixels
**Mode:** {img.mode}

### ✅ Image uploaded successfully!

The image has been received and is ready for disease detection analysis.
In a production system, this would be processed by a trained CNN model.
"""
                
                image_input.change(fn=analyze_image, inputs=[image_input], outputs=image_output)

    return interface

print("✓ Premium interface function ready")

## 🚀 Initialize & Launch System

In [ ]:
def initialize_system():
    """Initialize the RAG system with plant disease papers"""
    print("🔧 Initializing Plant Disease RAG System...")
    
    rag_system = PlantDiseaseRAG()
    papers = get_plant_disease_papers()
    rag_system.load_papers(papers)
    
    print(f"✅ System initialized with {len(papers)} papers")
    return rag_system

def find_available_port(start_port=7860, max_attempts=10):
    """Find available port to run the application"""
    import socket
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                return port
        except OSError:
            continue
    return None

print("✓ System initialization functions ready")

In [ ]:
def launch_app():
    """Launch the Plant Disease Monitoring application"""
    rag_system = initialize_system()
    
    if GRADIO_AVAILABLE:
        interface = create_premium_interface(rag_system)
        port = find_available_port()
        if port is None:
            interface.launch(share=True, show_error=False, quiet=True)
        else:
            interface.launch(share=True, server_port=port, show_error=False, quiet=True)
    else:
        print("⚠️ Gradio not available. System can be used programmatically.")
        return rag_system

# Run the app
if __name__ == "__main__":
    launch_app()

---
## 📝 Documentation

### System Architecture

```
┌─────────────────────────────────────────────────────────┐
│                  Presentation Layer                      │
│         (Gradio Premium UI with Dark Theme)              │
└─────────────────────────────────────────────────────────┘
                         ↕
┌─────────────────────────────────────────────────────────┐
│                   Application Layer                      │
│  • RAG Search Engine    • Anomaly Detection              │
│  • Sensor Processing    • Dashboard Generation           │
└─────────────────────────────────────────────────────────┘
                         ↕
┌─────────────────────────────────────────────────────────┐
│                    Data Layer                            │
│  • Vector Store (ChromaDB/Simple)                        │
│  • Embeddings (Transformer/TF-IDF)                       │
│  • Document Store                                        │
└─────────────────────────────────────────────────────────┘
                         ↕
┌─────────────────────────────────────────────────────────┐
│                 Cloud/IoT Layer                          │
│        Adafruit IO (MQTT Broker + Data Storage)          │
└─────────────────────────────────────────────────────────┘
```

### Features

1. **RAG Search Engine**: Semantic search using embeddings and vector store
2. **Real-time Sensors**: Live IoT data from Adafruit IO
3. **MQTT Query**: Historical data retrieval
4. **Dashboard**: Interactive Plotly visualizations
5. **Anomaly Detection**: Smart alert system
6. **Image Upload**: Plant image analysis interface

### Technical Stack

- **UI**: Gradio with custom dark theme
- **Embeddings**: Sentence Transformers / TF-IDF
- **Vector Store**: ChromaDB / SimpleVectorStore
- **Visualization**: Plotly
- **IoT**: Adafruit IO + MQTT

---

🌱 **Plant Disease Monitoring System - Premium Edition** 🌱